# middleschool-cardlist

Note: this notebook is only for introductory purposes, and is not the way our list is compiled anymore. See the README for more information.

## Prepare the data

Download raw data from [MTGJSON](https://mtgjson.com/) (uncomment and run only once)


In [1]:
# !cd data
# !wget "https://mtgjson.com/api/v5/AllPrintings.json.bz2"
# !bunzip2 AllPrintings.json.bz2
# !cd -

The Raw data is very large, so let's make JSON files for all relevant sets

Note: this cell can take a couple minutes to run


In [2]:
setlist = ['4ED', 'ICE', 'CHR', 'HML', 'ALL', 'MIR', 'VIS', '5ED',
           'WTH', 'POR', 'TMP', 'STH', 'EXO', 'P02', 'USG', 'ULG',
           '6ED', 'UDS', 'PTK', 'S99', 'MMQ', 'NEM', 'PCY', 'S00',
           'INV', 'PLS', '7ED', 'APC', 'ODY', 'TOR', 'JUD', 'ONS',
           'LGN', 'SCG', 'PDRC', 'PHPR', 'ATH', 'BRB', 'BTD', 'DKM']
for set in setlist:
    # Write a separate JSON document for each Middle School legal set
    command = 'cat data/AllPrintings.json | jq \'.data.\"' + \
        set + '\".cards\' > data/set_' + set + '.json'
    !{command}


Concatenate all set files into `middleschool.json`


In [3]:
command = "jq -s add data/set_* > data/middleschool.json"
!{command}


Create a list with each card's oracle ID, English name, and Japanese name


In [7]:
import json
import pandas as pd

with open("data/middleschool.json") as json_data:
    cards = json.loads(json_data.read())

# Create a pandas DataFrame with all cards from all legal sets
column_names = ["oracle_id", "name", "name_ja"]
middleschool_df = pd.DataFrame(columns=column_names)
for card in cards:
    oracle_id = card["identifiers"]["scryfallOracleId"]
    name = card["name"]
    lang_ja = [lang for lang in card["foreignData"] if lang["language"] == "Japanese"]
    # Some cards do not have a Japanese name
    if len(lang_ja) > 0:
        name_ja = lang_ja[0]["name"]
    else:
        name_ja = None
    temporary_df = pd.DataFrame(
        {"oracle_id": [oracle_id], "name": [name], "name_ja": [name_ja]}
    )
    middleschool_df = pd.concat([middleschool_df, temporary_df])

# For cards with multiple occurrences, put the rows that have the Japanese name on top
middleschool_df = middleschool_df.sort_values(by=["name", "name_ja"])
# For cards with multiple occurrences, delete all rows except for the top one
middleschool_df = middleschool_df.drop_duplicates(subset=["oracle_id"])
print(middleschool_df.shape[0], "cards found")
print("These are the first and last 5 cards")
print(middleschool_df.head())
print(middleschool_df.tail())

5800 cards found
These are the first and last 5 cards
                              oracle_id               name    name_ja
0  8adbba6e-03ef-4278-aec5-8a4496b377a8       Abandon Hope         断念
0  5a70ccfa-d12d-4e62-a1a4-f05cda2fd442  Abandoned Outpost  見捨てられた前哨地
0  c208b959-d0e4-4a9a-8255-2c7cc7596767    Abbey Gargoyles  修道院のガーゴイル
0  62e3f285-886c-414e-b4ff-403a7c01c23a       Abbey Matron       None
0  d0e1904e-1a37-41f6-8582-b9ea794bb886          Abduction         誘拐
                              oracle_id                      name    name_ja
0  ae8773a3-05f2-4074-9a53-033b0c127235  Zuo Ci, the Mocking Sage  嘲笑する仙人 左慈
0  c6eaa147-3566-43a9-999a-d58b877496f5            Zur's Weirding   ズアーの運命支配
0  ee0f883f-d7c9-4acf-a78f-f733b6f268d3           Zuran Enchanter       None
0  08cb8a30-9cb4-4517-bee5-8848aa60d1a2                 Zuran Orb       None
0  bc7b90b1-3517-4e5d-9bd8-68b4d8a259fd         Zuran Spellcaster       None


Remove Japanese card names that are wrong on MTGJSON


In [8]:
wrongnames = [
    "Aether Barrier",
    "Aether Burst",
    "Aether Charge",
    "Aether Flash",
    "Aether Mutation",
    "Aether Sting",
    "Aether Storm",
    "Aether Tide",
    "Tainted Aether",
    "Tar Pit Warrior",
]
print("Before:")
print(middleschool_df.loc[middleschool_df["name"].isin(wrongnames)])
middleschool_df.loc[middleschool_df["name"].isin(wrongnames), "name_ja"] = None
print("After:")
print(middleschool_df.loc[middleschool_df["name"].isin(wrongnames)])

Before:
                              oracle_id             name          name_ja
0  0fe602b7-9f88-4d3d-af24-7790df867ed5   Aether Barrier    Æther Barrier
0  1e33f39b-a61a-4a09-a541-16cc1bd53d02     Aether Burst      Æther Burst
0  15e83068-6253-4c65-8679-7295f3dc2075    Aether Charge     Æther Charge
0  a3c35742-e306-49b6-b042-db4f685c6f86     Aether Flash      Æther Flash
0  6697fe5b-90ac-4321-aa2f-cdc6ec283cb4  Aether Mutation  Aether Mutation
0  61105cb5-d7a1-4021-a006-dd1b947dfa68     Aether Sting      Æther Sting
0  ff4297d3-3d96-4bd6-a606-1bdc20a6df2b     Aether Storm      Æther Storm
0  2fbf95b4-bcf4-4b5e-b5dc-0294f2b48d3e      Aether Tide       Æther Tide
0  a61ceda1-5993-479e-945f-15753eeb7049   Tainted Aether    Tainted Æther
0  05a7ca83-e820-433f-b9e9-151e817d3708  Tar Pit Warrior  Tar Pit Warrior
After:
                              oracle_id             name name_ja
0  0fe602b7-9f88-4d3d-af24-7790df867ed5   Aether Barrier    None
0  1e33f39b-a61a-4a09-a541-16cc1bd53d02  

Find Japanese names for cards that were not released in Japanese in Middle School legal sets


In [10]:
import time
from requests_html import HTMLSession

session = HTMLSession()


def find_japanese_name(name):
    url = "http://whisper.wisdom-guild.net/card/" + name + "/"
    r = session.get(url)
    # Find the text on the <title> element in the HTML document
    title = r.html.find("title")[0].text
    # Find the position of the English card name within the title
    idx = title.find(name)
    # The Japanese name should be before the English name,
    # so if idx is 0, there is no Japanese name
    if idx == 0:
        return None
    # If the exact English card name can't be found, we look for a '/'
    if idx == -1:
        idx = title.find("/")
        # No '/' means no Japanese name
        if idx == -1:
            return None
        # Take only the Japanese name from the title
        name_ja = title[0:idx]
    else:
        # Take only the Japanese name from the title
        name_ja = title[0 : idx - 1]
    return name_ja


english_only_cards = middleschool_df[middleschool_df["name_ja"].isnull()]
name_list = english_only_cards["name"].to_list()
for idx, name in enumerate(name_list):
    middleschool_df.loc[middleschool_df["name"] == name, "name_ja"] = (
        find_japanese_name(name)
    )
    # print(middleschool_df.loc[middleschool_df['name'] == name])
    print(".", end="")
    if idx % 80 == 79:
        print()
    time.sleep(1)

................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
......................

Exclude all cards banned in Middle School


In [11]:
banlist = [
    "Amulet of Quoz",
    "Balance",
    "Brainstorm",
    "Bronze Tablet",
    "Channel",
    "Dark Ritual",
    "Demonic Consultation",
    "Flash",
    "Goblin Recruiter",
    "Imperial Seal",
    "Jeweled Bird",
    "Mana Crypt",
    "Mana Vault",
    "Memory Jar",
    "Mind's Desire",
    "Mind Twist",
    "Rebirth",
    "Strip Mine",
    "Tempest Efreet",
    "Timmerian Fiends",
    "Tolarian Academy",
    "Vampiric Tutor",
    "Windfall",
    "Yawgmoth's Bargain",
    "Yawgmoth's Will",
]
print("Cards legal by set:", middleschool_df.shape[0])
# Find the rows with the banned cards
banned_df = middleschool_df[
    pd.DataFrame(middleschool_df.name.tolist()).isin(banlist).any(axis=1).values
]
print("Banned cards:", banned_df.shape[0])
# Append the banned cards to the main Middle School DataFrame,
# then remove any rows that appear twice,
# effectively leaving only the legal cards
middleschool_df = pd.concat([middleschool_df, banned_df]).drop_duplicates(keep=False)
print("Cards legal by set and not banned:", middleschool_df.shape[0])
middleschool_df = middleschool_df.reset_index(drop=True)
middleschool_df = middleschool_df[["oracle_id", "name", "name_ja"]]

Cards legal by set: 5800
Banned cards: 25
Cards legal by set and not banned: 5775


Save the list to a CSV file and a JSON file


In [12]:
middleschool_df.to_csv("static/middleschool.csv")
middleschool_df.to_json("static/middleschool.json")

Feel free to delete everything in the `data` directory after you are done
